In [ ]:
#!git clone https://github.com/huggingface/transformers.git#


In [ ]:
#pip install transformers=="4.0.0"

In [ ]:
from fastai.text.all import *
import torch
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import pandas as pd
import glob

In [ ]:
pretrained_weights = 'gpt2'
tokenizer = GPT2TokenizerFast.from_pretrained(pretrained_weights)
model = GPT2LMHeadModel.from_pretrained(pretrained_weights)

In [ ]:
ids = tokenizer.encode('This is an example of text, and')
t = torch.LongTensor(ids)[None]
preds = model.generate(t)
tokenizer.decode(preds[0].numpy())


In [ ]:
with open('/kaggle/input/dataforlyrics/21pilots/train.csv', encoding="utf8", errors='ignore') as f:
    train=pd.Series(f,index=None)
with open('/kaggle/input/dataforlyrics/21pilots/valid.csv', encoding="utf8", errors='ignore') as f:
    valid=pd.Series(f,index=None)

In [ ]:
train.shape,valid.shape


In [ ]:
train=train.dropna()
valid=valid.dropna()
train.head(),valid.head()

In [ ]:
all_texts = np.concatenate([train.values, valid.values])

all_texts.shape

In [ ]:
def tokenize(text):
    toks = tokenizer.tokenize(text)
    return tensor(tokenizer.convert_tokens_to_ids(toks))

tokenized = [tokenize(t) for t in progress_bar(all_texts)]

class TransformersTokenizer(Transform):
    def __init__(self, tokenizer): self.tokenizer = tokenizer
    def encodes(self, x): 
        return x if isinstance(x, Tensor) else tokenize(x)
        
    def decodes(self, x): return TitledStr(self.tokenizer.decode(x.cpu().numpy()))

In [ ]:
bs,sl = 4,256
splits = [range_of(train), list(range(len(train), len(all_texts)))]
tls = TfmdLists(tokenized, TransformersTokenizer(tokenizer), splits=splits, dl_type=LMDataLoader)
dls = tls.dataloaders(bs=bs, seq_len=sl)

In [ ]:
dls.show_batch(max_n=2)

In [ ]:
class DropOutput(Callback):
    def after_pred(self): self.learn.pred = self.pred[0]

In [ ]:
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), cbs=[DropOutput], metrics=Perplexity()).to_fp16()

In [ ]:
learn.validate()

In [ ]:
learn.path=Path('/')
learn.lr_find()

In [ ]:
learn.fit_one_cycle(20, 1e-4)

In [ ]:
learn.recorder.plot_loss()

In [ ]:
prompt = "\n uh "
prompt_ids = tokenizer.encode(prompt)
inp = tensor(prompt_ids)[None].cuda()
preds = learn.model.generate(inp,
                             do_sample=True, 
                             max_length=90,
                             min_length=5,
                             top_k=40,
                             num_return_sequences=1)
tokenizer.decode(preds[0].cpu().tolist())


In [ ]:
learn.export('/kaggle/working/21pilots.pkl')

In [ ]:
#device=torch.device('cpu:0')
#learn.model.to(device)